In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_excel('online_retail.xlsx')

 reference date set to : 2011-12-15 (mid of month) 
 1. recency - 
 2. frequency
 3. monetary

In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
#droppping misisng values
df.dropna(axis = 0,how='any',inplace=True)

In [7]:
# Adding total price 
df['TotalPrice'] = df['Quantity'] + df['UnitPrice']

In [8]:
df.drop(columns = ['StockCode','Description','UnitPrice','Quantity'],inplace = True)

In [9]:
df.head()

,InvoiceNo,InvoiceDate,CustomerID,Country,TotalPrice
0,536365,2010-12-01 08:26:00,17850.0,United Kingdom,8.55
1,536365,2010-12-01 08:26:00,17850.0,United Kingdom,9.39
2,536365,2010-12-01 08:26:00,17850.0,United Kingdom,10.75
3,536365,2010-12-01 08:26:00,17850.0,United Kingdom,9.39
4,536365,2010-12-01 08:26:00,17850.0,United Kingdom,9.39


In [10]:
reference_date = pd.to_datetime('2011-12-15')

In [11]:
customer = df.groupby('CustomerID')
latest_transaction = customer['InvoiceDate'].max()
recency = (reference_date - latest_transaction).dt.days
recency

CustomerID
12346.0    330
12347.0      7
12348.0     80
12349.0     23
12350.0    315
          ... 
18280.0    282
18281.0    185
18282.0     12
18283.0      8
18287.0     47
Name: InvoiceDate, Length: 4372, dtype: int64

In [12]:
frequency = customer['InvoiceNo'].count()
frequency

CustomerID
12346.0      2
12347.0    182
12348.0     31
12349.0     73
12350.0     17
          ... 
18280.0     10
18281.0      7
18282.0     13
18283.0    756
18287.0     70
Name: InvoiceNo, Length: 4372, dtype: int64

In [13]:
monetary = customer['TotalPrice'].sum()
monetary

CustomerID
12346.0       2.08
12347.0    2939.21
12348.0    2519.71
12349.0    1236.10
12350.0     262.30
            ...   
18280.0      92.65
18281.0      93.36
18282.0     160.68
18283.0    2617.93
18287.0    1690.55
Name: TotalPrice, Length: 4372, dtype: float64

In [14]:
customer['TotalPrice'].transform('sum')

0         2917.51
1         2917.51
2         2917.51
3         2917.51
4         2917.51
           ...   
541904     632.17
541905     632.17
541906     632.17
541907     632.17
541908     632.17
Name: TotalPrice, Length: 406829, dtype: float64

In [15]:
rfm = pd.concat([recency, frequency,monetary], axis=1)
rfm.columns = ['R','F','M']
rfm

,R,F,M
CustomerID,,,
12346.0,330,2,2.08
12347.0,7,182,2939.21
12348.0,80,31,2519.71
12349.0,23,73,1236.10
12350.0,315,17,262.30
...,...,...,...
18280.0,282,10,92.65
18281.0,185,7,93.36
18282.0,12,13,160.68


In [16]:
rfm['R'] = pd.qcut(rfm['R'], 5, labels=[5, 4, 3, 2, 1])  # Lower Recency gets a higher score
rfm['F'] = pd.qcut(rfm['F'], 5, labels=[1, 2, 3, 4, 5])  # Higher Frequency gets a higher score
rfm['M'] = pd.qcut(rfm['M'], 5, labels=[1, 2, 3, 4, 5])   # Higher Monetary value gets a higher score

In [17]:
for x in rfm.columns:
    rfm[x] = rfm[x].astype('int64')

In [18]:
rfm['Score'] = rfm.mean(axis=1)

In [19]:
rfm

,R,F,M,Score
CustomerID,,,,
12346.0,1,1,1,1.000000
12347.0,5,5,5,5.000000
12348.0,2,3,5,3.333333
12349.0,4,4,4,4.000000
12350.0,1,2,2,1.666667
...,...,...,...,...
18280.0,1,1,1,1.000000
18281.0,1,1,1,1.000000
18282.0,5,1,1,2.333333


In [20]:


# Define segmentation rules based on R, F, M values
def rfm_segment(row):
    if row['R'] >= 4 and row['F'] >= 4 and row['M'] >= 4:
        return 'Champions'
    elif row['R'] >= 3 and row['F'] >= 3 and row['M'] >= 3:
        return 'Loyal Customers'
    elif row['R'] >= 4 and row['F'] <= 2:
        return 'Potential Loyalists'
    elif row['R'] <= 2 and row['F'] >= 4 and row['M'] >= 4:
        return 'At Risk'
    elif row['R'] <= 2 and row['F'] <= 2 and row['M'] <= 2:
        return 'Lost'
    else:
        return 'Need Attention'

# Apply the segmentation rules to the DataFrame
rfm['Segment'] = rfm.apply(rfm_segment, axis=1)

# Display the resulting DataFrame with customer segments
print(rfm)


            R  F  M     Score              Segment
CustomerID                                        
12346.0     1  1  1  1.000000                 Lost
12347.0     5  5  5  5.000000            Champions
12348.0     2  3  5  3.333333       Need Attention
12349.0     4  4  4  4.000000            Champions
12350.0     1  2  2  1.666667                 Lost
...        .. .. ..       ...                  ...
18280.0     1  1  1  1.000000                 Lost
18281.0     1  1  1  1.000000                 Lost
18282.0     5  1  1  2.333333  Potential Loyalists
18283.0     5  5  5  5.000000            Champions
18287.0     3  4  5  4.000000      Loyal Customers

[4372 rows x 5 columns]


# RFM Customer Segmentation
1. Champions: Customers who scored high (4 or 5) on all RFM metrics. These are your most valuable customers.
2. Loyal Customers: Customers who scored at least 3 on R, F, and M. They are good customers who engage with the business regularly.
3. Potential Loyalists: Customers with high recency (R ≥ 4) but low frequency (F ≤ 2), indicating they are recent buyers with the potential to become loyal.
4. At Risk: Customers with low recency (R ≤ 2), high frequency (F ≥ 4), and high monetary value (M ≥ 4), who used to be valuable but haven’t purchased recently.
5. Lost: Customers with low scores on all metrics (R, F, M ≤ 2), indicating that they have likely churned.
6. Need Attention: This is a catch-all for customers who don’t fit neatly into the other categories but still need engagement.

In [3]:
rfm['Segment'].value_counts()

Segment
Need Attention         1090
Champions               985
Lost                    908
Loyal Customers         824
Potential Loyalists     359
At Risk                 206
Name: count, dtype: int64

In [22]:
%store rfm

Stored 'rfm' (DataFrame)


In [1]:
%store -r rfm

In [4]:
rfm.to_csv('rfm.csv')